In [2]:
!pip install pandas

  Using cached pandas-2.2.0-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached pandas-2.2.0-cp310-cp310-win_amd64.whl (11.6 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
   ---------------------------------------- 0.0/345.4 kB ? eta -:--:--
   ---------------- ----------------------- 143.4/345.4 kB 4.2 MB/s eta 0:00:01
   -------------------- ------------------- 174.1/345.4 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------  337.9/345.4 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 345.4/345.4 kB 2.4 MB/s eta 0:00:00


In [3]:
import requests
import string
import json
import re
import pandas as pd
import creds


C:\Users\chaub\AppData\Local\Temp\ipykernel_37948\331663753.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
#Making file for caching and working on dataset, do not need to retain all the data
filename = 'APIcall.csv'
f1_file = open(filename, 'w')
colnames_1 = 'title, author, published_date, location, link, excerpt, summary, rank, topic, id, media\n'
f1_file.write(colnames_1)
f1_file.close()

#cols to save to permanent record to track user interests
f2_col_names = ['title', 'published_date', 'location', 'link']


In [16]:
endpoint = 'https://api.newscatcherapi.com/v2/search?'
places = ['Boulder, CO'] #, 'Longmont, CO', 'Colorado Springs, Colorado']
start_date = '2024/01/20'
end_date = '2024/02/05'
headers = {'x-api-key': creds.api_key}

#Add constraint using regex to avoid any searches with Reporting by Brad Brooks in Longmont, Colorado

for place in places:

    params = {
        'q': 'Longmont, Colorado',
        'lang': 'en',
        'countries': 'US',
        'ranked_only': True,
        'sort_by': 'rank',
        'page_size': 100,
        'page':1,
        'to': end_date,
        'from': start_date
    }

    response = requests.get(endpoint, headers=headers, params=params)
    print(response)
    json_text = response.json()
    #json_str = json.dumps(json_text, indent=4)
    print(json_str)

    f1_file = open(filename, 'a')
    
    for item in json_text['articles'][-5:]:

        #INSERT condition to check item['summary'] for regex
        #if present continue to next iteration
        regex_pattern = rf'Reporting by .{{0,50}} in {place}'
        if re.search(regex_pattern, item['summary']):
            continue

        
        #title, author, published_date, location, link, excerpt, summary, rank, topic, id, media
        #Get title
        title = item['title']
       
        #Get author
        author = item['author']

        #Get published_date
        published_date = item['published_date'][:10]

        #Get link
        link = item['link']

        #Get excerpt
        excerpt = item['excerpt']

        #Get summary
        summary = item['summary']

        #Get rank
        rank = item['rank']

        #Get topic
        topic = item['topic']

        #Get id
        id = item['_id']

        #Get media for thumbnail image
        media = item['media']

        #title, author, published_date, location, 
        #link, excerpt, summary, rank, topic, id, media
        write_row = str(title) +','+str(author)+','+str(published_date)+','\
                    +str(place)+','+str(link)+','+str(excerpt)+','\
                    +str(summary)+','+str(rank)+','+str(topic)+','\
                    +str(id)+','+str(media)+'\n'
        print(write_row)
        print("\n\nTHIS IS A NEW ENTRY\n\n")

        f1_file.write(write_row)

    f1_file.close()



<Response [200]>
{
    "status": "ok",
    "total_hits": 398,
    "page": 1,
    "total_pages": 4,
    "page_size": 100,
    "articles": [
        {
            "title": "Mrs. Longmont says she is 'not a pageant girl'",
            "author": "Dana Cadey",
            "published_date": "2024-01-27 15:40:00",
            "published_date_precision": "full",
            "link": "https://news.yahoo.com/mrs-longmont-says-she-not-154000745.html",
            "clean_url": "yahoo.com",
            "excerpt": "Jan. 27\u2014Michelle Deines is, in her own words, not a pageant girl. That's not stopping her from returning to the Mrs. Colorado contest for her second year in a row, representing her home city of\u2026",
            "summary": "Jan. 27\u2014Michelle Deines is, in her own words, not a pageant girl.\nThat's not stopping her from returning to the Mrs. Colorado contest for her second year in a row, representing her home city of Longmont.\n\"I'm a stay-at-home mom, and I am incredibly introv

In [25]:
#Several news articles appeared in the data because they had the following string:
#Reporting by (author) in Longmont, Colorado
#We want to remove all articles with the above string

#TEST
location = "Longmont, Colorado"

# Create the regex pattern with the location variable
regex_pattern = rf'Reporting by .{{0,50}} in {location}'


# Example usage:
text = "Some news article Reporting by Amrit singh in Longmont, Colorado"
if re.search(regex_pattern, text):
    print("Match found!")
else:
    print("No match found.")




Match found!
